In [1]:
from dotenv import load_dotenv 

In [2]:
load_dotenv()
import os

# Verify API key
if os.getenv("GOOGLE_API_KEY"):
    print("✅ GOOGLE_API_KEY found")
else:
    print("❌ GOOGLE_API_KEY not found - please set it in your .env file")

✅ GOOGLE_API_KEY found


In [4]:
# Core Imports

# Standard library
import numpy as np
import pandas as pd
import asyncio
import json

# LangChain components
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# RAGAS components
from ragas import SingleTurnSample, EvaluationDataset, evaluate
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    LLMContextPrecisionWithReference,
    LLMContextRecall,
    ContextEntityRecall,
    NoiseSensitivity
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

print("✅ All imports successful")

c:\Users\shara\OneDrive\Documents\Live Courses Krish Naik\R_A_G\Simple_rag\langchain\Lib\site-packages\instructor\providers\gemini\client.py:6: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


✅ All imports successful


In [5]:
!ollama list

NAME                     ID              SIZE      MODIFIED    
gemma3:1b                8648f39daa8f    815 MB    5 weeks ago    
nomic-embed-text:v1.5    0a109f422b47    274 MB    5 weeks ago    


In [5]:
from langchain_ollama import OllamaEmbeddings, ChatOllama

In [6]:
llm = ChatOllama(model="gemma3:1b", temperature=0)

In [7]:
# Initialize LLM and Embeddings

# Initialize base models  (OPENAI)

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:v1.5",
    # base_url="http://localhost:11434"  # Default Ollama URL
)


#  embeddings= GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Initialize base models (GEMINI)
#llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
#embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Wrap for RAGAS compatibility
ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

print("✅ LLM initialized: gpt-4o-mini")
print("✅ Embeddings initialized: text-embedding-3-small")
print("✅ RAGAS wrappers ready")

✅ LLM initialized: gpt-4o-mini
✅ Embeddings initialized: text-embedding-3-small
✅ RAGAS wrappers ready


C:\Users\shara\AppData\Local\Temp\ipykernel_22344\1223316257.py:18: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(llm)
C:\Users\shara\AppData\Local\Temp\ipykernel_22344\1223316257.py:19: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)


In [8]:
# Helper function for running async code in Jupyter

def run_async(coro):
    """Helper to run async code in Jupyter notebooks"""
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in Jupyter with an existing loop
            import nest_asyncio
            nest_asyncio.apply()
            return loop.run_until_complete(coro)
        else:
            return asyncio.run(coro)
    except RuntimeError:
        return asyncio.run(coro)

print("✅ Async helper ready")

✅ Async helper ready


In [9]:
# Define our test case

# The response we want to evaluate
test_response = "The first Super Bowl was held on January 15, 1967 in Los Angeles. It was a sunny day with clear skies."

# The context that was retrieved (source of truth)
test_context = [
    "The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles, California."
]

print("📝 Response to evaluate:")
print(f"   '{test_response}'")
print("\n📚 Retrieved context:")
print(f"   '{test_context[0]}'")

📝 Response to evaluate:
   'The first Super Bowl was held on January 15, 1967 in Los Angeles. It was a sunny day with clear skies.'

📚 Retrieved context:
   'The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles, California.'


In [10]:
# Manual claim extraction using LLM (mimicking RAGAS)

claim_extraction_prompt = ChatPromptTemplate.from_template("""
Given the following response, extract ALL factual claims as a numbered list.
Each claim should be a single, verifiable statement.

Response: {response}

Extract each factual claim:
""")

claim_chain = claim_extraction_prompt | llm | StrOutputParser()

extracted_claims_raw = claim_chain.invoke({"response": test_response})

print("🔍 STEP 1: Extracted Claims from Response")
print("=" * 50)
print(extracted_claims_raw)

🔍 STEP 1: Extracted Claims from Response
Here’s a numbered list of the factual claims extracted from the response:

1.  The first Super Bowl was held on January 15, 1967.
2.  It was a sunny day.
3.  It was a clear day. 




In [11]:
# Parse the claims into a list for verification

# For our analysis, let's define the claims explicitly
claims = [
    "The first Super Bowl was held on January 15, 1967",
    "The first Super Bowl was held in Los Angeles",
    "It was a sunny day",
    "There were clear skies"
]

print("📋 Claims to verify:")
for i, claim in enumerate(claims,1):
    print(f"   {i}. {claim}")

📋 Claims to verify:
   1. The first Super Bowl was held on January 15, 1967
   2. The first Super Bowl was held in Los Angeles
   3. It was a sunny day
   4. There were clear skies


In [12]:
# Manual claim verification (mimicking RAGAS)

verification_prompt = ChatPromptTemplate.from_template("""
Given the following context and claim, determine if the claim is SUPPORTED by the context.

Context: {context}

Claim: {claim}

Answer with:
- "SUPPORTED" if the claim can be verified from the context
- "NOT SUPPORTED" if the claim cannot be verified or contradicts the context

Also provide a brief explanation.

Verdict:
""")

verify_chain = verification_prompt | llm | StrOutputParser()

print("🔍 STEP 2: Verifying Each Claim Against Context")
print("=" * 60)

verification_results = []
for claim in claims:
    result = verify_chain.invoke({
        "context": test_context[0],
        "claim": claim
    })
    is_supported = "SUPPORTED" in result.upper() and "NOT SUPPORTED" not in result.upper()
    verification_results.append({
        "claim": claim,
        "supported": is_supported,
        "explanation": result
    })
    status = "✅" if is_supported else "❌"
    print(f"\n{status} Claim: '{claim}'")
    print(f"   Result: {result[:100]}..." if len(result) > 100 else f"   Result: {result}")

🔍 STEP 2: Verifying Each Claim Against Context

❌ Claim: 'The first Super Bowl was held on January 15, 1967'
   Result: Verdict: NOT SUPPORTED

Explanation: The context explicitly states the AFL-NFL World Championship Ga...

❌ Claim: 'The first Super Bowl was held in Los Angeles'
   Result: Verdict: NOT SUPPORTED

Explanation: The context explicitly states the AFL-NFL World Championship Ga...

❌ Claim: 'It was a sunny day'
   Result: Verdict: NOT SUPPORTED

Explanation: The context states the game was played on a specific date and l...

❌ Claim: 'There were clear skies'
   Result: Verdict: NOT SUPPORTED

Explanation: The context states the game was played on January 15, 1967, at ...


In [13]:
# Display verification results as a table

print("\n📊 Claim Verification Summary")
print("=" * 80)

df_verification = pd.DataFrame([
    {
        "Claim": r["claim"],
        "Supported?": "✅ Yes" if r["supported"] else "❌ No",
        "Reason": "Found in context" if r["supported"] else "HALLUCINATION - Not in context!"
    }
    for r in verification_results
])

print(df_verification.to_string(index=False))


📊 Claim Verification Summary
                                            Claim Supported?                          Reason
The first Super Bowl was held on January 15, 1967       ❌ No HALLUCINATION - Not in context!
     The first Super Bowl was held in Los Angeles       ❌ No HALLUCINATION - Not in context!
                               It was a sunny day       ❌ No HALLUCINATION - Not in context!
                           There were clear skies       ❌ No HALLUCINATION - Not in context!


In [14]:
# Manual faithfulness calculation

supported_count = sum(1 for r in verification_results if r["supported"])
total_claims = len(verification_results)

manual_faithfulness = supported_count / total_claims

print("🔢 STEP 3: Calculate Faithfulness Score")
print("=" * 50)
print(f"\n   Supported claims: {supported_count}")
print(f"   Total claims: {total_claims}")
print(f"\n   Formula: Faithfulness = {supported_count} / {total_claims}")
print(f"\n   📊 Manual Faithfulness Score: {manual_faithfulness:.2f}")

🔢 STEP 3: Calculate Faithfulness Score

   Supported claims: 0
   Total claims: 4

   Formula: Faithfulness = 0 / 4

   📊 Manual Faithfulness Score: 0.00


In [15]:
# Run actual RAGAS Faithfulness metric

# Create sample in RAGAS format
faithfulness_sample = SingleTurnSample(
    user_input="When was the first Super Bowl?",
    response=test_response,
    retrieved_contexts=test_context
)

# Initialize and run the metric
faithfulness_metric = Faithfulness(llm=ragas_llm)

ragas_faithfulness = run_async(faithfulness_metric.single_turn_ascore(faithfulness_sample))

print("🔬 RAGAS Faithfulness Result")
print("=" * 50)
print(f"\n   Manual calculation:  {manual_faithfulness:.2f}")
print(f"   RAGAS metric score:  {ragas_faithfulness:.2f}")
print(f"\n   Difference: {abs(manual_faithfulness - ragas_faithfulness):.2f}")

🔬 RAGAS Faithfulness Result

   Manual calculation:  0.00
   RAGAS metric score:  0.75

   Difference: 0.75


In [16]:
# Compare different faithfulness scenarios

faithfulness_examples = [
    {
        "name": "Perfect Faithfulness (No hallucinations)",
        "response": "The first Super Bowl was played on January 15, 1967 at the Los Angeles Memorial Coliseum.",
        "context": ["The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum."]
    },
    {
        "name": "Partial Faithfulness (Some hallucinations)",
        "response": "The first Super Bowl was on January 15, 1967. The Green Bay Packers won 35-10 with Bart Starr as MVP.",
        "context": ["The First AFL-NFL World Championship Game was played on January 15, 1967."]
    },
    {
        "name": "Zero Faithfulness (Complete hallucination)",
        "response": "The first Super Bowl was held in Miami in 1970 and attracted over 100,000 spectators.",
        "context": ["The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum."]
    }
]

print("📊 Faithfulness Comparison: Different Scenarios")
print("=" * 70)

for example in faithfulness_examples:
    sample = SingleTurnSample(
        user_input="Tell me about the first Super Bowl",
        response=example["response"],
        retrieved_contexts=example["context"]
    )
    score = run_async(faithfulness_metric.single_turn_ascore(sample))
    
    print(f"\n🏷️  {example['name']}")
    print(f"   Response: '{example['response'][:80]}...'" if len(example['response']) > 80 else f"   Response: '{example['response']}'")
    print(f"   Score: {score:.4f}")

📊 Faithfulness Comparison: Different Scenarios



🏷️  Perfect Faithfulness (No hallucinations)
   Response: 'The first Super Bowl was played on January 15, 1967 at the Los Angeles Memorial ...'
   Score: 1.0000

🏷️  Partial Faithfulness (Some hallucinations)
   Response: 'The first Super Bowl was on January 15, 1967. The Green Bay Packers won 35-10 wi...'
   Score: 1.0000

🏷️  Zero Faithfulness (Complete hallucination)
   Response: 'The first Super Bowl was held in Miami in 1970 and attracted over 100,000 specta...'
   Score: 1.0000


In [18]:
# Define our test case for relevancy

original_question = "When was the first Super Bowl?"
test_answer = "The first Super Bowl was held on January 15, 1967"

print("📝 Original Question:")
print(f"   '{original_question}'")
print("\n📝 Answer to Evaluate:")
print(f"   '{test_answer}'")

📝 Original Question:
   'When was the first Super Bowl?'

📝 Answer to Evaluate:
   'The first Super Bowl was held on January 15, 1967'


In [19]:
# Manual hypothetical question generation (mimicking RAGAS)

question_gen_prompt = ChatPromptTemplate.from_template("""
Given the following answer, generate exactly 3 different questions that this answer would be a good response to.
The questions should be varied but all answerable by this response.

Answer: {answer}

Generate 3 questions (one per line):
1.
2.
3.
""")

question_gen_chain = question_gen_prompt | llm | StrOutputParser()

generated_questions_raw = question_gen_chain.invoke({"answer": test_answer})

print("🔍 STEP 1: Generated Hypothetical Questions")
print("=" * 50)
print(generated_questions_raw)

🔍 STEP 1: Generated Hypothetical Questions
Here are 3 questions generated from the provided answer, suitable for a variety of levels of engagement:

1.  What year did the first Super Bowl take place?
2.  Besides being a sporting event, what significant event occurred on January 15, 1967?
3.  Can you name the year the Super Bowl was established?


In [20]:
# Parse generated questions (for our calculation)

# Manually define likely generated questions
generated_questions = [
    "When was the first Super Bowl held?",
    "What date was the inaugural Super Bowl?",
    "On what day did the first Super Bowl take place?"
]

print("📋 Questions for embedding comparison:")
print(f"   Original: '{original_question}'")
print("   Generated:")
for i, q in enumerate(generated_questions, 1):
    print(f"      {i}. '{q}'")

📋 Questions for embedding comparison:
   Original: 'When was the first Super Bowl?'
   Generated:
      1. 'When was the first Super Bowl held?'
      2. 'What date was the inaugural Super Bowl?'
      3. 'On what day did the first Super Bowl take place?'


In [21]:
# Define cosine similarity function

def cosine_similarity(vec1, vec2):
    """Calculate cosine similarity between two vectors"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("✅ Cosine similarity function ready")
print("\n📐 Formula: cos(θ) = (A · B) / (||A|| × ||B||)")

✅ Cosine similarity function ready

📐 Formula: cos(θ) = (A · B) / (||A|| × ||B||)


In [22]:
# Calculate embeddings and similarities

print("🔍 STEP 2: Computing Embeddings and Similarities")
print("=" * 60)

# Get embedding for original question
original_embedding = embeddings.embed_query(original_question)
print(f"\n✅ Original question embedded (dim={len(original_embedding)})")

# Get embeddings for generated questions and calculate similarities
similarities = []
for i, gen_q in enumerate(generated_questions, 1):
    gen_embedding = embeddings.embed_query(gen_q)
    sim = cosine_similarity(original_embedding, gen_embedding)
    similarities.append(sim)
    print(f"\n   Question {i}: '{gen_q}'")
    print(f"   Similarity to original: {sim:.4f}")

🔍 STEP 2: Computing Embeddings and Similarities

✅ Original question embedded (dim=768)

   Question 1: 'When was the first Super Bowl held?'
   Similarity to original: 0.9588

   Question 2: 'What date was the inaugural Super Bowl?'
   Similarity to original: 0.9097

   Question 3: 'On what day did the first Super Bowl take place?'
   Similarity to original: 0.9229


In [23]:
# Calculate answer relevancy score

manual_relevancy = np.mean(similarities)

print("🔢 STEP 3: Calculate Answer Relevancy Score")
print("=" * 50)
print(f"\n   Similarities: {[f'{s:.4f}' for s in similarities]}")
print(f"   Formula: Average of similarities")
print(f"\n   ({' + '.join([f'{s:.4f}' for s in similarities])}) / {len(similarities)}")
print(f"\n   📊 Manual Answer Relevancy: {manual_relevancy:.4f}")

🔢 STEP 3: Calculate Answer Relevancy Score

   Similarities: ['0.9588', '0.9097', '0.9229']
   Formula: Average of similarities

   (0.9588 + 0.9097 + 0.9229) / 3

   📊 Manual Answer Relevancy: 0.9305


In [24]:
# Run actual RAGAS Answer Relevancy metric

relevancy_sample = SingleTurnSample(
    user_input=original_question,
    response=test_answer,
    retrieved_contexts=["The First AFL-NFL World Championship Game was played on January 15, 1967."]
)

relevancy_metric = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

ragas_relevancy = run_async(relevancy_metric.single_turn_ascore(relevancy_sample))

print("🔬 RAGAS Answer Relevancy Result")
print("=" * 50)
print(f"\n   Manual calculation:  {manual_relevancy:.4f}")
print(f"   RAGAS metric score:  {ragas_relevancy:.4f}")

🔬 RAGAS Answer Relevancy Result

   Manual calculation:  0.9305
   RAGAS metric score:  0.9627


In [25]:
# Compare different relevancy scenarios

relevancy_examples = [
    {
        "name": "Highly Relevant (Directly answers WHEN)",
        "question": "When was the first Super Bowl?",
        "answer": "The first Super Bowl was held on January 15, 1967.",
    },
    {
        "name": "Partially Relevant (Answers but adds extra info)",
        "question": "When was the first Super Bowl?",
        "answer": "The Super Bowl is the annual championship game of the NFL, first held on January 15, 1967.",
    },
    {
        "name": "Low Relevancy (Doesn't answer WHEN)",
        "question": "When was the first Super Bowl?",
        "answer": "The Super Bowl is the annual championship game of the National Football League.",
    },
    {
        "name": "Off-topic (Completely irrelevant)",
        "question": "When was the first Super Bowl?",
        "answer": "Pizza is a popular Italian dish that spread worldwide in the 20th century.",
    }
]

print("📊 Answer Relevancy Comparison")
print("=" * 70)

for example in relevancy_examples:
    sample = SingleTurnSample(
        user_input=example["question"],
        response=example["answer"],
        retrieved_contexts=["Context not relevant for this metric."]
    )
    score = run_async(relevancy_metric.single_turn_ascore(sample))
    
    print(f"\n🏷️  {example['name']}")
    print(f"   Q: '{example['question']}'")
    print(f"   A: '{example['answer'][:60]}...'" if len(example['answer']) > 60 else f"   A: '{example['answer']}'")
    print(f"   Score: {score:.4f}")

📊 Answer Relevancy Comparison

🏷️  Highly Relevant (Directly answers WHEN)
   Q: 'When was the first Super Bowl?'
   A: 'The first Super Bowl was held on January 15, 1967.'
   Score: 0.9632

🏷️  Partially Relevant (Answers but adds extra info)
   Q: 'When was the first Super Bowl?'
   A: 'The Super Bowl is the annual championship game of the NFL, f...'
   Score: 0.7506

🏷️  Low Relevancy (Doesn't answer WHEN)
   Q: 'When was the first Super Bowl?'
   A: 'The Super Bowl is the annual championship game of the Nation...'
   Score: 0.7506

🏷️  Off-topic (Completely irrelevant)
   Q: 'When was the first Super Bowl?'
   A: 'Pizza is a popular Italian dish that spread worldwide in the...'
   Score: 0.3138


In [26]:
# Define our test case for context precision

question = "Where is the Eiffel Tower located?"
reference = "The Eiffel Tower is located in Paris, France."

# Chunks with known relevance
chunks_with_relevance = [
    ("The Eiffel Tower is located in Paris, France.", True),      # Directly relevant
    ("Paris is the capital of France.", True),                     # Somewhat relevant
    ("The tower was built in 1889.", False),                       # Not relevant to WHERE
    ("Pizza originated in Italy.", False),                         # Completely irrelevant
]

print("📝 Question: '{}'\n".format(question))
print("📚 Retrieved Chunks (with relevance):")
for i, (chunk, relevant) in enumerate(chunks_with_relevance, 1):
    status = "✅ Relevant" if relevant else "❌ Not relevant"
    print(f"   {i}. {status}: '{chunk}'")

📝 Question: 'Where is the Eiffel Tower located?'

📚 Retrieved Chunks (with relevance):
   1. ✅ Relevant: 'The Eiffel Tower is located in Paris, France.'
   2. ✅ Relevant: 'Paris is the capital of France.'
   3. ❌ Not relevant: 'The tower was built in 1889.'
   4. ❌ Not relevant: 'Pizza originated in Italy.'


In [27]:
# Manual relevance classification using LLM

relevance_prompt = ChatPromptTemplate.from_template("""
Given the question and reference answer, determine if the following context chunk is RELEVANT.

Question: {question}
Reference Answer: {reference}
Context Chunk: {chunk}

Is this chunk relevant for answering the question? Answer only "RELEVANT" or "NOT RELEVANT".
""")

relevance_chain = relevance_prompt | llm | StrOutputParser()

print("🔍 Manual Relevance Classification")
print("=" * 60)

relevance_results = []
for chunk, expected in chunks_with_relevance:
    result = relevance_chain.invoke({
        "question": question,
        "reference": reference,
        "chunk": chunk
    })
    is_relevant = "RELEVANT" in result.upper() and "NOT RELEVANT" not in result.upper()
    relevance_results.append(is_relevant)
    status = "✅" if is_relevant else "❌"
    print(f"{status} '{chunk[:50]}...' → {result.strip()}")

🔍 Manual Relevance Classification
✅ 'The Eiffel Tower is located in Paris, France....' → RELEVANT
❌ 'Paris is the capital of France....' → NOT RELEVANT
❌ 'The tower was built in 1889....' → NOT RELEVANT
❌ 'Pizza originated in Italy....' → NOT RELEVANT


In [28]:
# Calculate Precision@K for GOOD ranking (relevant at top)

# Good ranking: [Relevant, Relevant, Not Relevant, Not Relevant]
good_ranking = [True, True, False, False]

print("📊 GOOD RANKING: Relevant chunks at TOP")
print("=" * 60)
print("\nRanking: [✅ Relevant, ✅ Relevant, ❌ Not Rel, ❌ Not Rel]")
print("\nPrecision@K calculation:")

precisions_good = []
relevant_count = 0
for k, is_relevant in enumerate(good_ranking, 1):
    if is_relevant:
        relevant_count += 1
    precision_at_k = relevant_count / k
    contributes = "→ Contributes" if is_relevant else "→ Does NOT contribute"
    print(f"   Position {k}: Precision@{k} = {relevant_count}/{k} = {precision_at_k:.2f} {contributes}")
    if is_relevant:
        precisions_good.append(precision_at_k)

total_relevant = sum(good_ranking)
context_precision_good = sum(precisions_good) / total_relevant if total_relevant > 0 else 0

print(f"\n   Sum of contributing precisions: {sum(precisions_good):.2f}")
print(f"   Total relevant items: {total_relevant}")
print(f"\n   📊 Context Precision (Good Ranking): {context_precision_good:.2f}")

📊 GOOD RANKING: Relevant chunks at TOP

Ranking: [✅ Relevant, ✅ Relevant, ❌ Not Rel, ❌ Not Rel]

Precision@K calculation:
   Position 1: Precision@1 = 1/1 = 1.00 → Contributes
   Position 2: Precision@2 = 2/2 = 1.00 → Contributes
   Position 3: Precision@3 = 2/3 = 0.67 → Does NOT contribute
   Position 4: Precision@4 = 2/4 = 0.50 → Does NOT contribute

   Sum of contributing precisions: 2.00
   Total relevant items: 2

   📊 Context Precision (Good Ranking): 1.00


In [30]:
# Calculate Precision@K for BAD ranking (relevant at bottom)

# Bad ranking: [Not Relevant, Not Relevant, Relevant, Relevant]
bad_ranking = [False, False, True, True]

print("📊 BAD RANKING: Relevant chunks at BOTTOM")
print("=" * 60)
print("\nRanking: [❌ Not Rel, ❌ Not Rel, ✅ Relevant, ✅ Relevant]")
print("\nPrecision@K calculation:")

precisions_bad = []
relevant_count = 0
for k, is_relevant in enumerate(bad_ranking, 1):
    if is_relevant:
        relevant_count += 1
    precision_at_k = relevant_count / k
    contributes = "→ Contributes" if is_relevant else "→ Does NOT contribute"
    print(f"   Position {k}: Precision@{k} = {relevant_count}/{k} = {precision_at_k:.2f} {contributes}")
    if is_relevant:
        precisions_bad.append(precision_at_k)

total_relevant = sum(bad_ranking)
context_precision_bad = sum(precisions_bad) / total_relevant if total_relevant > 0 else 0

print(f"\n   Sum of contributing precisions: {sum(precisions_bad):.2f}")
print(f"   Total relevant items: {total_relevant}")
print(f"\n   📊 Context Precision (Bad Ranking): {context_precision_bad:.2f}")

📊 BAD RANKING: Relevant chunks at BOTTOM

Ranking: [❌ Not Rel, ❌ Not Rel, ✅ Relevant, ✅ Relevant]

Precision@K calculation:
   Position 1: Precision@1 = 0/1 = 0.00 → Does NOT contribute
   Position 2: Precision@2 = 0/2 = 0.00 → Does NOT contribute
   Position 3: Precision@3 = 1/3 = 0.33 → Contributes
   Position 4: Precision@4 = 2/4 = 0.50 → Contributes

   Sum of contributing precisions: 0.83
   Total relevant items: 2

   📊 Context Precision (Bad Ranking): 0.42


In [31]:
# Visual comparison

print("\n" + "=" * 60)
print("📊 RANKING COMPARISON")
print("=" * 60)

print("""
GOOD RANKING (Score: {:.2f})          BAD RANKING (Score: {:.2f})
┌─────────────────────────┐          ┌─────────────────────────┐
│ 1. ✅ Eiffel Tower Paris │          │ 1. ❌ Pizza Italy        │
│ 2. ✅ Paris is capital   │          │ 2. ❌ Built in 1889      │
│ 3. ❌ Built in 1889      │          │ 3. ✅ Paris is capital   │
│ 4. ❌ Pizza Italy        │          │ 4. ✅ Eiffel Tower Paris │
└─────────────────────────┘          └─────────────────────────┘
     Relevant at TOP! ✓                  Relevant at BOTTOM! ✗
""".format(context_precision_good, context_precision_bad))

print(f"   Difference: {context_precision_good - context_precision_bad:.2f}")
print("   Same chunks, different ranking → HUGE difference in score!")


📊 RANKING COMPARISON

GOOD RANKING (Score: 1.00)          BAD RANKING (Score: 0.42)
┌─────────────────────────┐          ┌─────────────────────────┐
│ 1. ✅ Eiffel Tower Paris │          │ 1. ❌ Pizza Italy        │
│ 2. ✅ Paris is capital   │          │ 2. ❌ Built in 1889      │
│ 3. ❌ Built in 1889      │          │ 3. ✅ Paris is capital   │
│ 4. ❌ Pizza Italy        │          │ 4. ✅ Eiffel Tower Paris │
└─────────────────────────┘          └─────────────────────────┘
     Relevant at TOP! ✓                  Relevant at BOTTOM! ✗

   Difference: 0.58
   Same chunks, different ranking → HUGE difference in score!


In [32]:
# Run actual RAGAS Context Precision

# Good ranking sample
good_sample = SingleTurnSample(
    user_input=question,
    reference=reference,
    retrieved_contexts=[
        "The Eiffel Tower is located in Paris, France.",
        "Paris is the capital of France.",
        "The tower was built in 1889.",
        "Pizza originated in Italy."
    ]
)

# Bad ranking sample (same chunks, reversed order)
bad_sample = SingleTurnSample(
    user_input=question,
    reference=reference,
    retrieved_contexts=[
        "Pizza originated in Italy.",
        "The tower was built in 1889.",
        "Paris is the capital of France.",
        "The Eiffel Tower is located in Paris, France."
    ]
)

precision_metric = LLMContextPrecisionWithReference(llm=ragas_llm)

good_score = run_async(precision_metric.single_turn_ascore(good_sample))
bad_score = run_async(precision_metric.single_turn_ascore(bad_sample))

print("🔬 RAGAS Context Precision Results")
print("=" * 50)
print(f"\n   Good Ranking (relevant at top): {good_score:.2f}")
print(f"   Bad Ranking (relevant at bottom): {bad_score:.2f}")
print(f"\n   Difference: {good_score - bad_score:.2f}")

🔬 RAGAS Context Precision Results

   Good Ranking (relevant at top): 1.00
   Bad Ranking (relevant at bottom): 1.00

   Difference: 0.00


In [33]:
# Context Recall example setup

recall_question = "Tell me about the Eiffel Tower."
recall_reference = "The Eiffel Tower is located in Paris. It was built in 1889. It is 330 meters tall."

# Retrieved context (missing the height information)
recall_context = [
    "The Eiffel Tower is a landmark located in Paris, France.",
    "The tower was completed in 1889 for the World's Fair."
]

print("📝 Reference Answer (Ground Truth):")
print(f"   '{recall_reference}'")
print("\n📚 Retrieved Context:")
for i, ctx in enumerate(recall_context, 1):
    print(f"   {i}. '{ctx}'")

📝 Reference Answer (Ground Truth):
   'The Eiffel Tower is located in Paris. It was built in 1889. It is 330 meters tall.'

📚 Retrieved Context:
   1. 'The Eiffel Tower is a landmark located in Paris, France.'
   2. 'The tower was completed in 1889 for the World's Fair.'


In [34]:
# Extract claims from reference

reference_claims = [
    "The Eiffel Tower is located in Paris",
    "It was built in 1889",
    "It is 330 meters tall"
]

print("🔍 STEP 1: Reference Claims")
print("=" * 50)
for i, claim in enumerate(reference_claims, 1):
    print(f"   {i}. {claim}")

🔍 STEP 1: Reference Claims
   1. The Eiffel Tower is located in Paris
   2. It was built in 1889
   3. It is 330 meters tall


In [35]:
# Check attribution of each claim

attribution_prompt = ChatPromptTemplate.from_template("""
Can the following claim be attributed to (found in) the given context?

Context:
{context}

Claim: {claim}

Answer "YES" if the claim is supported by the context, "NO" if it cannot be found.
""")

attribution_chain = attribution_prompt | llm | StrOutputParser()

print("🔍 STEP 2: Claim Attribution Check")
print("=" * 60)

combined_context = "\n".join(recall_context)
attribution_results = []

for claim in reference_claims:
    result = attribution_chain.invoke({
        "context": combined_context,
        "claim": claim
    })
    found = "YES" in result.upper()
    attribution_results.append(found)
    status = "✅ Found" if found else "❌ MISSING"
    print(f"   {status}: '{claim}'")
    if not found:
        print(f"      ⚠️ This information was NOT retrieved!")

🔍 STEP 2: Claim Attribution Check
   ✅ Found: 'The Eiffel Tower is located in Paris'
   ✅ Found: 'It was built in 1889'
   ✅ Found: 'It is 330 meters tall'


In [36]:
# Calculate Context Recall

claims_found = sum(attribution_results)
total_claims = len(reference_claims)
manual_recall = claims_found / total_claims

print("🔢 STEP 3: Calculate Context Recall")
print("=" * 50)
print(f"\n   Claims found in context: {claims_found}")
print(f"   Total claims in reference: {total_claims}")
print(f"\n   Formula: {claims_found} / {total_claims} = {manual_recall:.2f}")
print(f"\n   📊 Context Recall: {manual_recall:.2f}")
print(f"\n   ⚠️ Interpretation: {100 - manual_recall*100:.0f}% of required info was NOT retrieved!")

🔢 STEP 3: Calculate Context Recall

   Claims found in context: 3
   Total claims in reference: 3

   Formula: 3 / 3 = 1.00

   📊 Context Recall: 1.00

   ⚠️ Interpretation: 0% of required info was NOT retrieved!


In [37]:
# Verify with RAGAS

recall_sample = SingleTurnSample(
    user_input=recall_question,
    response="The Eiffel Tower is in Paris and was built in 1889.",
    reference=recall_reference,
    retrieved_contexts=recall_context
)

recall_metric = LLMContextRecall(llm=ragas_llm)
ragas_recall = run_async(recall_metric.single_turn_ascore(recall_sample))

print("🔬 RAGAS Context Recall Result")
print("=" * 50)
print(f"\n   Manual calculation: {manual_recall:.2f}")
print(f"   RAGAS metric score: {ragas_recall:.2f}")

🔬 RAGAS Context Recall Result

   Manual calculation: 1.00
   RAGAS metric score: 1.00


In [38]:
# Entity Recall example setup

entity_reference = "Albert Einstein developed the theory of relativity at Princeton University in 1905."
entity_context = [
    "Albert Einstein was a famous physicist who worked at Princeton."
]

print("📝 Reference Answer:")
print(f"   '{entity_reference}'")
print("\n📚 Retrieved Context:")
print(f"   '{entity_context[0]}'")

📝 Reference Answer:
   'Albert Einstein developed the theory of relativity at Princeton University in 1905.'

📚 Retrieved Context:
   'Albert Einstein was a famous physicist who worked at Princeton.'


In [39]:
# Manual entity extraction

entity_extraction_prompt = ChatPromptTemplate.from_template("""
Extract all named entities from the following text. 
Include: PERSON, ORGANIZATION, LOCATION, DATE, and other proper nouns.

Text: {text}

List each entity on a new line with its type:
""")

entity_chain = entity_extraction_prompt | llm | StrOutputParser()

print("🔍 Entity Extraction")
print("=" * 60)

print("\n📋 Reference Entities:")
ref_entities = entity_chain.invoke({"text": entity_reference})
print(ref_entities)

print("\n📋 Context Entities:")
ctx_entities = entity_chain.invoke({"text": entity_context[0]})
print(ctx_entities)

🔍 Entity Extraction

📋 Reference Entities:
Here’s a breakdown of the named entities extracted from the text, with their types:

*   Albert Einstein – PERSON
*   Princeton University – ORGANIZATION
*   1905 – DATE


📋 Context Entities:
Here’s the extracted named entities from the text, with their types:

*   Albert Einstein – PERSON
*   physicist – ORGANIZATION
*   Princeton – LOCATION



In [40]:
# Manual entity analysis

# Define entities for analysis
reference_entities = {
    "Albert Einstein": "PERSON",
    "Princeton University": "ORGANIZATION",
    "1905": "DATE"
}

context_entities = {
    "Albert Einstein": "PERSON",
    "Princeton": "ORGANIZATION"  # Partial match
}

print("📊 Entity Comparison")
print("=" * 60)

print("\n| Entity in Reference | Type | Found in Context? |")
print("|" + "-" * 20 + "|" + "-" * 14 + "|" + "-" * 18 + "|")

found_count = 0
for entity, entity_type in reference_entities.items():
    # Check if entity (or partial) exists in context
    found = any(entity.lower() in ctx.lower() or ctx.lower() in entity.lower() 
                for ctx in context_entities.keys())
    if found:
        found_count += 1
    status = "✅ Yes" if found else "❌ MISSING"
    print(f"| {entity:18} | {entity_type:12} | {status:16} |")

entity_recall = found_count / len(reference_entities)
print(f"\n📊 Entity Recall: {found_count}/{len(reference_entities)} = {entity_recall:.2f}")
print(f"⚠️ Missing: '1905' - Critical date not retrieved!")

📊 Entity Comparison

| Entity in Reference | Type | Found in Context? |
|--------------------|--------------|------------------|
| Albert Einstein    | PERSON       | ✅ Yes            |
| Princeton University | ORGANIZATION | ✅ Yes            |
| 1905               | DATE         | ❌ MISSING        |

📊 Entity Recall: 2/3 = 0.67
⚠️ Missing: '1905' - Critical date not retrieved!


In [41]:
# Verify with RAGAS

entity_sample = SingleTurnSample(
    reference=entity_reference,
    retrieved_contexts=entity_context
)

entity_metric = ContextEntityRecall(llm=ragas_llm)
ragas_entity_recall = run_async(entity_metric.single_turn_ascore(entity_sample))

print("🔬 RAGAS Context Entity Recall Result")
print("=" * 50)
print(f"\n   Manual estimate: {entity_recall:.2f}")
print(f"   RAGAS metric score: {ragas_entity_recall:.2f}")

🔬 RAGAS Context Entity Recall Result

   Manual estimate: 0.67
   RAGAS metric score: 0.25


In [42]:
# Noise Sensitivity example setup

noise_question = "What is LIC known for?"
noise_response = "LIC is the largest insurance company in India, known for its vast portfolio. LIC contributes to financial stability."
noise_reference = "LIC is the largest insurance company in India, established in 1956. It is known for managing a large portfolio of investments."

noise_contexts = [
    "LIC was established in 1956 following nationalization.",           # ✅ Relevant
    "LIC is the largest insurance company with huge investments.",      # ✅ Relevant
    "LIC manages substantial funds for financial stability.",           # ✅ Relevant
    "The Indian economy is one of the fastest-growing economies..."     # ❌ NOISE!
]

print("📝 Question: '{}'\n".format(noise_question))
print("📝 Response to evaluate:")
print(f"   '{noise_response}'")
print("\n📝 Reference (Ground Truth):")
print(f"   '{noise_reference}'")
print("\n📚 Retrieved Contexts:")
for i, ctx in enumerate(noise_contexts, 1):
    noise_tag = " ← NOISE!" if i == 4 else " ✅"
    print(f"   {i}. '{ctx[:60]}...'{noise_tag}")

📝 Question: 'What is LIC known for?'

📝 Response to evaluate:
   'LIC is the largest insurance company in India, known for its vast portfolio. LIC contributes to financial stability.'

📝 Reference (Ground Truth):
   'LIC is the largest insurance company in India, established in 1956. It is known for managing a large portfolio of investments.'

📚 Retrieved Contexts:
   1. 'LIC was established in 1956 following nationalization....' ✅
   2. 'LIC is the largest insurance company with huge investments....' ✅
   3. 'LIC manages substantial funds for financial stability....' ✅
   4. 'The Indian economy is one of the fastest-growing economies.....' ← NOISE!


In [43]:
# Analyze claims in response

response_claims = [
    ("LIC is the largest insurance company in India", True, "Matches reference"),
    ("LIC is known for its vast portfolio", True, "Matches reference (portfolio)"),
    ("LIC contributes to financial stability", False, "NOT in reference - possible hallucination from noise!")
]

print("🔍 Claim Analysis")
print("=" * 70)

print("\n| Claim | Correct? | Reason |")
print("|" + "-" * 45 + "|" + "-" * 10 + "|" + "-" * 40 + "|")

incorrect_count = 0
for claim, is_correct, reason in response_claims:
    status = "✅ Yes" if is_correct else "❌ No"
    if not is_correct:
        incorrect_count += 1
    print(f"| {claim[:43]:43} | {status:8} | {reason[:38]:38} |")

🔍 Claim Analysis

| Claim | Correct? | Reason |
|---------------------------------------------|----------|----------------------------------------|
| LIC is the largest insurance company in Ind | ✅ Yes    | Matches reference                      |
| LIC is known for its vast portfolio         | ✅ Yes    | Matches reference (portfolio)          |
| LIC contributes to financial stability      | ❌ No     | NOT in reference - possible hallucinat |


In [44]:
# Calculate Noise Sensitivity

total_claims = len(response_claims)
noise_sensitivity = incorrect_count / total_claims

print("🔢 Noise Sensitivity Calculation")
print("=" * 50)
print(f"\n   Incorrect claims: {incorrect_count}")
print(f"   Total claims: {total_claims}")
print(f"\n   Formula: {incorrect_count} / {total_claims} = {noise_sensitivity:.2f}")
print(f"\n   📊 Noise Sensitivity: {noise_sensitivity:.2f}")

if noise_sensitivity < 0.3:
    print("   ✅ Good! Model is mostly resistant to noise.")
elif noise_sensitivity < 0.6:
    print("   ⚠️ Warning! Model is sometimes confused by noise.")
else:
    print("   🚨 Bad! Model is highly susceptible to noise.")

🔢 Noise Sensitivity Calculation

   Incorrect claims: 1
   Total claims: 3

   Formula: 1 / 3 = 0.33

   📊 Noise Sensitivity: 0.33
   ⚠️ Warning! Model is sometimes confused by noise.


In [45]:
# Verify with RAGAS (both modes)

noise_sample = SingleTurnSample(
    user_input=noise_question,
    response=noise_response,
    reference=noise_reference,
    retrieved_contexts=noise_contexts
)

# Relevant mode: errors from relevant contexts
noise_metric_relevant = NoiseSensitivity(llm=ragas_llm, mode="relevant")

ragas_noise = run_async(noise_metric_relevant.single_turn_ascore(noise_sample))

print("🔬 RAGAS Noise Sensitivity Result")
print("=" * 50)
print(f"\n   Mode: relevant")
print(f"   Score: {ragas_noise:.2f}")
print(f"\n   Remember: Lower is better for this metric!")

🔬 RAGAS Noise Sensitivity Result

   Mode: relevant
   Score: 0.00

   Remember: Lower is better for this metric!
